In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler 
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from collections import Counter
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, classification_report

In [27]:
data=pd.read_csv('data.csv')

In [28]:
data =  data[data.columns].replace({'Yes':1, 'No':0, 'Male':1,'Female':0,'No, borderline diabetes':'0','Yes (during pregnancy)':'1' })
data['Diabetic'] = data['Diabetic'].astype(int)

/var/folders/dz/tmzcb2_j20bbp4pjqn9xwlvw0000gn/T/ipykernel_23534/2979708335.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data =  data[data.columns].replace({'Yes':1, 'No':0, 'Male':1,'Female':0,'No, borderline diabetes':'0','Yes (during pregnancy)':'1' })


In [29]:
data.columns

Index(['HeartDisease', 'BMI', 'Smoking', 'AlcoholDrinking', 'Stroke',
       'PhysicalHealth', 'MentalHealth', 'DiffWalking', 'Sex', 'AgeCategory',
       'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'SleepTime',
       'Asthma', 'KidneyDisease', 'SkinCancer'],
      dtype='object')

In [30]:
data['MentalHealth'].fillna(data['MentalHealth'].mean(),inplace=True)
selected_features = [
    'AgeCategory', 'DiffWalking', 'Diabetic', 'PhysicalHealth', 'Stroke',
    'Smoking','GenHealth',
    'Sex', 'Race', 'BMI', 'SkinCancer','HeartDisease', 'Asthma','MentalHealth'
]
X = data[selected_features]
y = data['KidneyDisease']
# X = X.drop(columns=['MentalHealth'], axis=1)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

/var/folders/dz/tmzcb2_j20bbp4pjqn9xwlvw0000gn/T/ipykernel_23534/332075556.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['MentalHealth'].fillna(data['MentalHealth'].mean(),inplace=True)


In [31]:
age_order = ['18-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80 or older']
ordinal_encoder = OrdinalEncoder(categories=[age_order])
X_train['AgeCategory'] = ordinal_encoder.fit_transform(X_train[['AgeCategory']])
X_val['AgeCategory'] = ordinal_encoder.transform(X_val[['AgeCategory']])
X_test['AgeCategory'] = ordinal_encoder.transform(X_test[['AgeCategory']])

In [32]:
#One-hot encode 'AgeCategory', 'Race', 'GenHealth' for all datasets
X_train = pd.get_dummies(X_train, columns=['Race', 'GenHealth'], drop_first=True)
X_val = pd.get_dummies(X_val, columns=['Race', 'GenHealth'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['Race', 'GenHealth'], drop_first=True)

In [33]:
# # balance the classes in KidneyDisease using SMOTE
# sm = SMOTE(random_state=42)
# X_train, y_train = sm.fit_resample(X_train, y_train)
# check the balance of the classes
print(Counter(y_train))

Counter({0: 32824, 1: 2616})


In [34]:
X_train, X_val = X_train.align(X_val, join='left', axis=1, fill_value=0)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [35]:
logreg = LogisticRegression(max_iter=500, random_state=42)
logreg.fit(X_train, y_train)
y_train_pred = logreg.predict(X_train)
y_val_pred = logreg.predict(X_val)
y_test_pred = logreg.predict(X_test)

In [36]:
print("Accuracy on training set: ", logreg.score(X_train, y_train))
print("Accuracy on validation set: ", logreg.score(X_val, y_val))
print("Accuracy on test set: ", logreg.score(X_test, y_test))

print("\nClassification Report (Train):\n", classification_report(y_train, y_train_pred))
print("\nConfusion Matrix (Train):\n", confusion_matrix(y_train, y_train_pred))
print("\nClassification Report (Validation):\n", classification_report(y_val, y_val_pred))
print("\nConfusion Matrix (Validation):\n", confusion_matrix(y_val, y_val_pred))
print("\nClassification Report (Test):\n", classification_report(y_test, y_test_pred))
print("\nConfusion Matrix (Test):\n", confusion_matrix(y_test, y_test_pred))


Accuracy on training set:  0.9260722347629797
Accuracy on validation set:  0.9234806162180463
Accuracy on test set:  0.9264432029795159

Classification Report (Train):
               precision    recall  f1-score   support

           0       0.93      1.00      0.96     32824
           1       0.42      0.00      0.01      2616

    accuracy                           0.93     35440
   macro avg       0.67      0.50      0.48     35440
weighted avg       0.89      0.93      0.89     35440


Confusion Matrix (Train):
 [[32809    15]
 [ 2605    11]]

Classification Report (Validation):
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     10910
           1       0.50      0.00      0.01       904

    accuracy                           0.92     11814
   macro avg       0.71      0.50      0.48     11814
weighted avg       0.89      0.92      0.89     11814


Confusion Matrix (Validation):
 [[10906     4]
 [  900     4]]

Classification 

In [37]:
svm_model = SVC(C=10, gamma=0.001, kernel='rbf', random_state=42)
svm_model.fit(X_train, y_train)
y_train_pred = svm_model.predict(X_train)
y_val_pred = svm_model.predict(X_val)
y_test_pred = svm_model.predict(X_test)

In [38]:
# get accuracy 
print("Accuracy on training set: ", svm_model.score(X_train, y_train))
print("Accuracy on validation set: ", svm_model.score(X_val, y_val))
print("Accuracy on test set: ", svm_model.score(X_test, y_test))

print("\nClassification Report (Train):\n", classification_report(y_train, y_train_pred))
print("\nConfusion Matrix (Train):\n", confusion_matrix(y_train, y_train_pred))
print("\nClassification Report (Validation):\n", classification_report(y_val, y_val_pred))
print("\nConfusion Matrix (Validation):\n", confusion_matrix(y_val, y_val_pred))
print("\nClassification Report (Test):\n", classification_report(y_test, y_test_pred))
print("\nConfusion Matrix (Test):\n", confusion_matrix(y_test, y_test_pred))

Accuracy on training set:  0.9261851015801355
Accuracy on validation set:  0.9234806162180463
Accuracy on test set:  0.9261892669713899

Classification Report (Train):
               precision    recall  f1-score   support

           0       0.93      1.00      0.96     32824
           1       0.00      0.00      0.00      2616

    accuracy                           0.93     35440
   macro avg       0.46      0.50      0.48     35440
weighted avg       0.86      0.93      0.89     35440


Confusion Matrix (Train):
 [[32824     0]
 [ 2616     0]]

Classification Report (Validation):
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     10910
           1       0.00      0.00      0.00       904

    accuracy                           0.92     11814
   macro avg       0.46      0.50      0.48     11814
weighted avg       0.85      0.92      0.89     11814


Confusion Matrix (Validation):
 [[10910     0]
 [  904     0]]

Classification 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  